This Notebook is the code which was used to generate the model


Link to Google Drive where the data file has been placed

In [0]:
from google.colab import drive
drive.mount('/content/gdrive') 

Use the Pandas Library to read the data csv file. And Install Geohash2

In [1]:
import pandas as pd
df = pd.read_csv("gdrive/My Drive/GrabAI/testing.csv")
!pip install git+https://github.com/dbarthe/geohash/
import geohash2

ModuleNotFoundError: No module named 'pandas'

Sort Dataframe by Day and TimeStamp

In [0]:
df.sort_values(by=['day','timestamp'], inplace=True)

Calculate Time in mins. A random id is assigned to each fragment for progress tracking.





In [0]:
import time
tic = time.time()
import numpy as np
from multiprocessing import Pool
import random

def parallelize_dataframe(df, func):
  fragment_array = np.array_split(df, 20)
  pool = Pool(20)
  df = pd.concat(pool.map(func, fragment_array))
  pool.close()
  pool.join()
  return df

def get_time(x):
  hour, minute = x[1].split(':')
  return int(x[0]-1)*24*60 + int(hour)*60 + int(minute)

def test_func(data):
  fragment_id = random.randint(0,999)
  print('Running: {}'.format(fragment_id))
  data['time'] = data[['day','timestamp']].apply(get_time, axis=1)
  print('Done: {}'.format(fragment_id))
  return data

df = parallelize_dataframe(df, test_func)
toc = time.time()
print(toc - tic)

Write  df to Pickle File

In [0]:
import pickle
outfile = open("df.pickle","wb")
pickle.dump(df, outfile)
outfile.close()

Retrieve df from Pickle File

In [0]:
import pickle
infile = open("df.pickle","rb")
df = pickle.load(infile)
infile.close()

Prepare Main Time Series Data Frame

In [0]:
import time
tic = time.time()
import numpy as np
from multiprocessing import Pool, Manager
import random

unique_geohash_list = list(np.unique(df['geohash6']))
n_unique_geohash = len(unique_geohash_list)
geohash_dict = {unique_geohash_list[i]:i for i in range(n_unique_geohash)}
max_time = np.amax(df['time'])
time_list = list(range(0, max_time, 15))
demand_df = pd.DataFrame(time_list, columns=['time'])
processes = 40

def parallelize_dataframe(df, func):
  fragment_array = np.array_split(df, processes)
  pool = Pool(processes)
  df = pd.concat(pool.map(func, fragment_array))
  pool.close()
  pool.join()
  return df

def get_demand_snapshot(x):
  demand_snapshot = np.zeros(n_unique_geohash)
  time_df = df['time'] == x
  df_snapshot = df[time_df]
  for _, row in df_snapshot.iterrows():
    demand_snapshot[geohash_dict[row['geohash6']]] = row['demand']
  return demand_snapshot

def test_func(data):
  fragment_id = random.randint(0,999)
  print('Running: {}'.format(fragment_id))
  data['demand_snapshot'] = data['time'].apply(get_demand_snapshot)
  n_done[0] += 1
  print('Done: {}, {}/{}'.format(fragment_id, n_done[0], processes))
  return data

n_done = Manager().list(range(1))
n_done[0] = 0
demand_df = parallelize_dataframe(demand_df, test_func)
toc = time.time()
print(toc - tic)



Restart package

In [0]:
import numpy as np
import pandas as pd
import pickle
infile = open("df.pickle","rb")
df = pickle.load(infile)
infile = open("demand_df.pickle","rb")
demand_df = pickle.load(infile)
infile.close()
infile.close()
unique_geohash_list = list(np.unique(df['geohash6']))
n_unique_geohash = len(unique_geohash_list)

Write  demand_df to Pickle File

In [0]:
import pickle
outfile = open("demand_df.pickle","wb")
pickle.dump(demand_df, outfile)
outfile.close()

Retrieve  demand_df from Pickle File

In [0]:
import pickle
infile = open("demand_df.pickle","rb")
demand_df = pickle.load(infile)
infile.close()

Data Preparation

In [0]:
import time
tic = time.time()

total_time = 30
training_df = demand_df['demand_snapshot'][demand_df['time'] <= (total_time)*24*60]

data_list= training_df.values.tolist()
data_arr= np.empty([len(data_list),0])
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = n_unique_geohash
  df = pd.DataFrame(data)
  global data_arr

  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    data_arr = np.append(data_arr, df.shift(i).values, axis=1)
    print('Input Sequence {}/{} Left'.format(i,n_in))
  
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    data_arr = np.append(data_arr, df.shift(-i).values, axis=1)

  print('Putting it together...')
  # drop rows with NaN values
  data_arr = data_arr[~np.isnan(data_arr).any(axis=1)]
    
  return
print('Working on Forecast Sequence...')
series_to_supervised(data_list, 96, 6)
print(data_arr.shape)

toc = time.time()
print(toc - tic)

Training and Testing Data Set Preparation

In [0]:
training_days = 25
values = data_arr
n_features = n_unique_geohash
n_time_step = 96
n_obs = int((n_time_step+1) * n_features)
n_total_time = training_days * 24 * 4
train = values[:n_total_time, :]
test = values[n_total_time:, :]
print(train.shape, test.shape)
train_X, train_y = train[:, :n_obs], train[:, n_obs:n_obs + n_features*5]
test_X, test_y = test[:, :n_obs], test[:, n_obs:n_obs + n_features*5]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
train_X = train_X.reshape((train_X.shape[0], n_time_step+1, n_features))
test_X = test_X.reshape((test_X.shape[0], n_time_step+1, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Recurrent Neural Network Modeling

In [0]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
import tensorflow as tf

# design network
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(256, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(6645,activation=tf.nn.relu))
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=256, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Get T+1 to T+5 values and place in DataFrame

In [0]:
yhat = model.predict(test_X)
names = list()
predictions_df = pd.DataFrame()
predictions_df['geohash'] = unique_geohash_list
for i in range(0,5):
  start = n_unique_geohash*i
  end = (i+1)*n_unique_geohash
  demand = yhat[0][start:end]
  predictions_df['T + {}'.format(i+1)] = demand
  

Calculate RMSE

In [0]:
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error

rmse = np.zeros(5)
# calculate RMSE
for i in range(0,5):
  start = n_unique_geohash*i
  end = (i+1)*n_unique_geohash
  rmse[i] = sqrt(mean_squared_error(test_y[0,start:end], yhat[0,start:end]))

print('RMSE for \nT+15: {}\nT+30: {}\nT+45: {}\nT+60: {}\nT+75: {}'.format(*rmse))

RMSE for 
T+15: 0.017088448323991992
T+30: 0.020636314772696267
T+45: 0.01905477288018001
T+60: 0.019645188305588854
T+75: 0.022823923811942613


In [0]:
model.save("gdrive/My Drive/GrabAI/model_30_96_25.h5")